## Libs

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
import math

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


## Functions

In [ ]:
## SWAPS

def swap_hist(timestamp_max):
    sample_transport=RequestsHTTPTransport(
       url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
       verify=True,
       retries=10,
    )
    client = Client(
       transport=sample_transport
    )
    query = gql('''
    query {
    swaps(orderBy: timestamp, orderDirection: desc, where:
     { pair: "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
       timestamp_lte: ''' + timestamp_max + '''}
    ) {

        id
        logIndex
        transaction
        pair {
           token0 {
             symbol
           }
           token1 {
             symbol
           }
         }
         amount0In
         amount0Out
         amount1In
         amount1Out
         amountUSD
         to
         timestamp

     }
    }
    ''')
    return client.execute(query)['swaps']


## MINT

def mint_hist(timestamp_max):
    sample_transport=RequestsHTTPTransport(
       url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
       verify=True,
       retries=10,
    )
    client = Client(
       transport=sample_transport
    )
    query = gql('''
    query {
    mints(orderBy: timestamp, orderDirection: desc, where:
     { pair: "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
       timestamp_lte: ''' + timestamp_max + '''}
    ) {

        id
        logIndex
        transaction
        timestamp
        pair {
           token0 {
             symbol
           }
           token1 {
             symbol
           }
         }
        to
        liquidity
        sender
        amount0
        amount1
        amountUSD
        feeTo
        feeLiquidity


     }
    }
    ''')
    return client.execute(query)['mints']
    
    
## BURN

def burn_hist(timestamp_max):
    sample_transport=RequestsHTTPTransport(
       url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
       verify=True,
       retries=10,
    )
    client = Client(
       transport=sample_transport
    )
    query = gql('''
    query {
    burns(orderBy: timestamp, orderDirection: desc, where:
     { pair: "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
       timestamp_lte: ''' + timestamp_max + '''}
    ) {

        id
        logIndex
        transaction
        timestamp
        pair {
           token0 {
             symbol
           }
           token1 {
             symbol
           }
         }
        to
        liquidity
        sender
        amount0
        amount1
        amountUSD
        feeTo
        feeLiquidity



     }
    }
    ''')
    return client.execute(query)['burns']
    
    
## PAIR

def pair_hist(timestamp_max):
    sample_transport=RequestsHTTPTransport(
       url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2',
       verify=True,
       retries=10,
    )
    client = Client(
       transport=sample_transport
    )
    query = gql('''
    query {
    pair(where:
     { pair: "0xa478c2975ab1ea89e8196811f51a7b7ade33eb11",
       timestamp: ''' + timestamp_max + '''}
    ) {

        id
        transaction
        timestamp
        pair {
           token0 {
             symbol
           }
           token1 {
             symbol
           }
         }
        to
        liquidity
        sender
        amount0
        amount1
        amountUSD
        feeTo
        feeLiquidity


     }
    }
    ''')
    return client.execute(query)['burns']

In [ ]:
def treat_dict_pair0(pair):

    return pair['token0']['symbol']

def treat_dict_pair1(pair):
    
    return pair['token1']['symbol']

In [ ]:
timestamp_initial = "1612000000"
timestamp_final = "1400000000"


## Swap historical data

In [ ]:
swap_list = []

timestamp_actual = timestamp_initial
int_timestamp = int(timestamp_actual)

while int_timestamp > int(timestamp_final):
    
    list_aux = swap_hist(timestamp_actual)
    if list_aux == []:
        break
    else:
        for item in list_aux:
            swap_list.append(item)

        if timestamp_actual == swap_list[-1]['timestamp']:
            break
        else:
            timestamp_actual = swap_list[-1]['timestamp']
            int_timestamp = int(timestamp_actual)



In [ ]:
df_swap = pd.DataFrame(swap_list)

df_swap['token0'] = df_swap['pair'].apply(lambda x: treat_dict_pair0(x))
df_swap['token1'] = df_swap['pair'].apply(lambda x: treat_dict_pair1(x))

df_swap = df_swap[['amount0In', 'amount0Out', 'amount1In', 'amount1Out', 'amountUSD', 'id', 'logIndex',
       'timestamp', 'to', 'token0', 'token1']].drop_duplicates()

## Mint historical data

In [ ]:
mint_list = []

timestamp_actual = timestamp_initial
int_timestamp = int(timestamp_actual)
while int_timestamp > int(timestamp_final):
    
    list_aux = mint_hist(timestamp_actual)
    if list_aux == []:
        break
    else:
        for item in list_aux:
            mint_list.append(item)

        if timestamp_actual == mint_list[-1]['timestamp']:
            break
        else:
            timestamp_actual = mint_list[-1]['timestamp']
            int_timestamp = int(timestamp_actual)


In [ ]:
df_mint = pd.DataFrame(mint_list)

df_mint['token0'] = df_mint['pair'].apply(lambda x: treat_dict_pair0(x))
df_mint['token1'] = df_mint['pair'].apply(lambda x: treat_dict_pair1(x))

df_mint = df_mint[['amount0', 'amount1', 'amountUSD', 'feeLiquidity', 'feeTo', 'id', 'logIndex',
       'liquidity', 'sender', 'timestamp', 'to',
       'token0', 'token1']].drop_duplicates()

## Burn historical data

In [ ]:
burn_list = []

timestamp_actual = timestamp_initial
int_timestamp = int(timestamp_actual)
while int_timestamp > int(timestamp_final):
    
    list_aux = burn_hist(timestamp_actual)
    if list_aux == []:
        break
    else:
        for item in list_aux:
            burn_list.append(item)
        
        if timestamp_actual == burn_list[-1]['timestamp']:
            break
        else:
            timestamp_actual = burn_list[-1]['timestamp']
            int_timestamp = int(timestamp_actual)


In [ ]:
df_burn = pd.DataFrame(burn_list)

df_burn['token0'] = df_burn['pair'].apply(lambda x: treat_dict_pair0(x))
df_burn['token1'] = df_burn['pair'].apply(lambda x: treat_dict_pair1(x))

df_burn = df_burn[['amount0', 'amount1', 'amountUSD', 'feeLiquidity', 'feeTo', 'id', 'logIndex',
       'liquidity', 'sender', 'timestamp', 'to',
       'token0', 'token1']].drop_duplicates()

## Saving historical data

In [ ]:
df_burn.to_csv('burn_historical_data.csv')
df_mint.to_csv('mint_historical_data.csv')
df_swap.to_csv('swap_historical_data.csv')

## Treating data to run cadCAD model

In [ ]:
df_swap_uniq = df_swap.copy()
df_swap_uniq['amount0In'] = df_swap_uniq['amount0In'].astype(float)
df_swap_uniq['amount0Out'] = df_swap_uniq['amount0Out'].astype(float)
df_swap_uniq['amount1In'] = df_swap_uniq['amount1In'].astype(float)
df_swap_uniq['amount1Out'] = df_swap_uniq['amount1Out'].astype(float)
df_swap_uniq['amountUSD'] = df_swap_uniq['amountUSD'].astype(float)

In [ ]:
df_swap_uniq['time'] = df_swap_uniq['timestamp'].apply(lambda x: datetime.utcfromtimestamp(int(x)))
df_swap_uniq = df_swap_uniq.sort_values(by=['time','logIndex']).reset_index(drop=True)

In [ ]:
df_swap_uniq = df_swap_uniq[['timestamp', 'amount0In', 'amount0Out', 'amount1In', 'amount1Out', 'amountUSD', 'time','logIndex']]
df_swap_uniq['token0'] = 'DAI'
df_swap_uniq['token1'] = 'WETH'

df_swap_uniq['amount0'] = df_swap_uniq['amount0In'] - df_swap_uniq['amount0Out']
df_swap_uniq['amount1'] = df_swap_uniq['amount1In'] - df_swap_uniq['amount1Out']

df_swap_uniq['amount0_pool_burn'] = 0
df_swap_uniq['amount1_pool_burn'] = 0

df_swap_uniq['amount0_pool_mint'] = 0
df_swap_uniq['amount1_pool_mint'] = 0

df_swap_uniq['liquidity_mint'] = 0
df_swap_uniq['liquidity_burn'] = 0

df_swap_uniq.head()

In [ ]:
df_mint_uniq = df_mint.copy()

df_mint_uniq = df_mint_uniq[['amount0', 'amount1', 'amountUSD','liquidity', 'timestamp', 'logIndex']]

df_mint_uniq['amount0'] = df_mint_uniq['amount0'].astype(float)
df_mint_uniq['amount1'] = df_mint_uniq['amount1'].astype(float)
df_mint_uniq['amountUSD'] = df_mint_uniq['amountUSD'].astype(float)
df_mint_uniq['liquidity'] = df_mint_uniq['liquidity'].astype(float)

#df_mint_uniq = df_mint_uniq.groupby(['timestamp']).sum().reset_index()
df_mint_uniq.sort_values(by=['timestamp','logIndex'],inplace=True)

df_mint_uniq['time'] = df_mint_uniq['timestamp'].apply(lambda x: datetime.utcfromtimestamp(int(x)))

df_mint_uniq['token0'] = 'DAI'
df_mint_uniq['token1'] = 'WETH'

df_mint_uniq['amount0In'] = 0
df_mint_uniq['amount0Out'] = 0
df_mint_uniq['amount1In'] = 0
df_mint_uniq['amount1Out'] = 0

df_mint_uniq['amount0_pool_burn'] = 0
df_mint_uniq['amount1_pool_burn'] = 0

df_mint_uniq['liquidity_mint'] = df_mint_uniq['liquidity']
df_mint_uniq['liquidity_burn'] = 0

df_mint_uniq['amount0_pool_mint'] = df_mint_uniq['amount0']
df_mint_uniq['amount1_pool_mint'] = df_mint_uniq['amount1']

df_mint_uniq = df_mint_uniq.loc[:,df_mint_uniq.columns != 'liquidity']

In [ ]:
df_mint_uniq

In [ ]:
df_burn_uniq = df_burn.copy()

df_burn_uniq = df_burn_uniq[['amount0', 'amount1', 'amountUSD','liquidity', 'timestamp', 'logIndex']]

df_burn_uniq['amount0'] = df_burn_uniq['amount0'].astype(float)
df_burn_uniq['amount1'] = df_burn_uniq['amount1'].astype(float)
df_burn_uniq['amountUSD'] = df_burn_uniq['amountUSD'].astype(float)
df_burn_uniq['liquidity'] = df_burn_uniq['liquidity'].astype(float)

#df_burn_uniq = df_burn_uniq.groupby(['timestamp']).sum().reset_index()
df_burn_uniq.sort_values(by=['timestamp','logIndex'],inplace=True)

df_burn_uniq['time'] = df_burn_uniq['timestamp'].apply(lambda x: datetime.utcfromtimestamp(int(x)))

df_burn_uniq['token0'] = 'DAI'
df_burn_uniq['token1'] = 'WETH'

df_burn_uniq['amount0In'] = 0
df_burn_uniq['amount0Out'] = 0
df_burn_uniq['amount1In'] = 0
df_burn_uniq['amount1Out'] = 0

df_burn_uniq['amount0_pool_mint'] = 0
df_burn_uniq['amount1_pool_mint'] = 0

df_burn_uniq['amount0_pool_burn'] = -df_burn_uniq['amount0']
df_burn_uniq['amount1_pool_burn'] = -df_burn_uniq['amount1']

df_burn_uniq['liquidity_mint'] = 0
df_burn_uniq['liquidity_burn'] = df_burn_uniq['liquidity']

df_burn_uniq['amount0'] = -df_burn_uniq['amount0']
df_burn_uniq['amount1'] = -df_burn_uniq['amount1']

df_burn_uniq = df_burn_uniq.loc[:,df_burn_uniq.columns != 'liquidity']

In [ ]:
# SWAP - Trades

df_swap_uniq_cad = df_swap_uniq.copy()
df_swap_uniq_cad = df_swap_uniq_cad.rename(columns={'time':'block_timestamp', 'amount0':'token_delta','amount1':'eth_delta'})
df_swap_uniq_cad['event'] = df_swap_uniq_cad['token_delta'].apply(lambda x: 'TokenPurchase' if x < 0 else 'EthPurchase')
df_swap_uniq_cad['contract_event'] = df_swap_uniq_cad['token_delta'].apply(lambda x: 'DAITokenPurchase' if x < 0 else 'DAIEthPurchase')
df_swap_uniq_cad['uni_delta'] = 0.
df_swap_uniq_cad['eth_balance'] = df_swap_uniq_cad['eth_delta']
df_swap_uniq_cad['token_balance'] = df_swap_uniq_cad['token_delta']
df_swap_uniq_cad['UNI_supply'] = 0.

df_swap_uniq_cad = df_swap_uniq_cad[['logIndex','block_timestamp','event', 'contract_event', 'eth_delta', 'token_delta',
                                     'uni_delta', 'eth_balance', 'token_balance', 'UNI_supply']]

In [ ]:
# MINT liquidity

df_mint_uniq_cad = df_mint_uniq.copy()
df_mint_uniq_cad = df_mint_uniq_cad.rename(columns={'time':'block_timestamp', 'amount0':'token_delta','amount1':'eth_delta'})
df_mint_uniq_cad['event'] = 'AddLiquidity'
df_mint_uniq_cad['contract_event'] = 'DAIAddLiquidity'
df_mint_uniq_cad['uni_delta'] = 0.
df_mint_uniq_cad['eth_balance'] = df_mint_uniq_cad['eth_delta']
df_mint_uniq_cad['token_balance'] = df_mint_uniq_cad['token_delta']
df_mint_uniq_cad['UNI_supply'] = 0.

df_mint_uniq_cad = df_mint_uniq_cad[['logIndex','block_timestamp','event', 'contract_event', 'eth_delta', 'token_delta',
                                     'uni_delta', 'eth_balance', 'token_balance', 'UNI_supply']]


# MINT LP transfer

df_mint_uniq_cad_2 = df_mint_uniq.copy()
df_mint_uniq_cad_2 = df_mint_uniq_cad_2.rename(columns={'time':'block_timestamp', 'liquidity_mint':'uni_delta'})
df_mint_uniq_cad_2['event'] = 'Transfer'
df_mint_uniq_cad_2['contract_event'] = 'DAITransfer'
df_mint_uniq_cad_2['eth_delta'] = 0.
df_mint_uniq_cad_2['token_delta'] = 0.
df_mint_uniq_cad_2['eth_balance'] = 0.
df_mint_uniq_cad_2['token_balance'] = 0.
df_mint_uniq_cad_2['UNI_supply'] = df_mint_uniq_cad_2['uni_delta']

df_mint_uniq_cad_2 = df_mint_uniq_cad_2[['logIndex','block_timestamp','event', 'contract_event', 'eth_delta', 'token_delta',
                                     'uni_delta', 'eth_balance', 'token_balance', 'UNI_supply']]


df_mint_uniq_cad = df_mint_uniq_cad.append(df_mint_uniq_cad_2)

In [ ]:
# BURN liquidity

df_burn_uniq_cad = df_burn_uniq.copy()
df_burn_uniq_cad = df_burn_uniq_cad.rename(columns={'time':'block_timestamp', 'amount0':'token_delta','amount1':'eth_delta'})
df_burn_uniq_cad['event'] = 'RemoveLiquidity'
df_burn_uniq_cad['contract_event'] = 'DAIRemoveLiquidity'
df_burn_uniq_cad['uni_delta'] = 0.
df_burn_uniq_cad['eth_balance'] = df_burn_uniq_cad['eth_delta']
df_burn_uniq_cad['token_balance'] = df_burn_uniq_cad['token_delta']
df_burn_uniq_cad['UNI_supply'] = 0.

df_burn_uniq_cad = df_burn_uniq_cad[['logIndex','block_timestamp','event', 'contract_event', 'eth_delta', 'token_delta',
                                     'uni_delta', 'eth_balance', 'token_balance', 'UNI_supply']]


# BURN LP transfer

df_burn_uniq_cad_2 = df_burn_uniq.copy()
df_burn_uniq_cad_2 = df_burn_uniq_cad_2.rename(columns={'time':'block_timestamp', 'liquidity_burn':'uni_delta'})
df_burn_uniq_cad_2['event'] = 'Transfer'
df_burn_uniq_cad_2['contract_event'] = 'DAITransfer'
df_burn_uniq_cad_2['eth_delta'] = 0.
df_burn_uniq_cad_2['token_delta'] = 0.
df_burn_uniq_cad_2['eth_balance'] = 0.
df_burn_uniq_cad_2['token_balance'] = 0.
df_burn_uniq_cad_2['uni_delta'] = -df_burn_uniq_cad_2['uni_delta']
df_burn_uniq_cad_2['UNI_supply'] = df_burn_uniq_cad_2['uni_delta']

df_burn_uniq_cad_2 = df_burn_uniq_cad_2[['logIndex','block_timestamp','event', 'contract_event', 'eth_delta', 'token_delta',
                                     'uni_delta', 'eth_balance', 'token_balance', 'UNI_supply']]


df_burn_uniq_cad = df_burn_uniq_cad.append(df_burn_uniq_cad_2)


In [ ]:
df_mint_uniq.head()

In [ ]:
## JOIN ETH_DAI database

df_ETHDAI_cad = df_swap_uniq_cad.append(df_mint_uniq_cad)
df_ETHDAI_cad = df_ETHDAI_cad.append(df_burn_uniq_cad)

df_ETHDAI_cad.sort_values(by=['block_timestamp','logIndex'],ascending=True,inplace=True)
df_ETHDAI_cad.reset_index(drop=True,inplace=True)

In [ ]:
def func_int(x):

    integ = x * 1e18
    
    return int(integ)

In [ ]:
df_ETHDAI_cad['eth_delta'] = df_ETHDAI_cad['eth_delta'].apply(lambda x: func_int(x))
df_ETHDAI_cad['token_delta'] = df_ETHDAI_cad['token_delta'].apply(lambda x: func_int(x))
df_ETHDAI_cad['uni_delta'] = df_ETHDAI_cad['uni_delta'].apply(lambda x: func_int(x))
df_ETHDAI_cad['eth_balance'] = df_ETHDAI_cad['eth_balance'].apply(lambda x: func_int(x))
df_ETHDAI_cad['token_balance'] = df_ETHDAI_cad['token_balance'].apply(lambda x: func_int(x))
df_ETHDAI_cad['UNI_supply'] = df_ETHDAI_cad['UNI_supply'].apply(lambda x: func_int(x))

In [ ]:
df_ETHDAI_cad['eth_balance'] = df_ETHDAI_cad['eth_balance'].cumsum()
df_ETHDAI_cad['token_balance'] = df_ETHDAI_cad['token_balance'].cumsum()
df_ETHDAI_cad['UNI_supply'] = df_ETHDAI_cad['UNI_supply'].cumsum()

In [ ]:
df_ETHDAI_cad.sort_values(by=['block_timestamp','logIndex'],ascending=True,inplace=True)

In [ ]:
df_ETHDAI_cad.to_csv('database_Uniswap_cadCAD.csv')
df_ETHDAI_cad.to_pickle('model/parts/uniswap_events.pickle')